In [1]:
# psh270

import numpy as np
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv")

In [2]:
data.set_index('Unnamed: 0', inplace=True)
